In [11]:
%load_ext autoreload
%autoreload 2

In [16]:
from data import dataloader
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [131]:
pilot_data = dataloader.break_out_blocks(
    dataloader.load_from_firebase()
)

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [129]:
for target in list(pilot_data.target.unique()):
    ib =max(
        pilot_data.loc[
            pilot_data.target==target
        ].iteration_block.dropna()
    )
    print(": ".join([
        target,
        "{0} on-target samples in {1} iteration block{2}".format(
            str(len(pilot_data.loc[
                (pilot_data.target == target) &
                (pilot_data.ontarget)
            ])),
            "%.0f" % ib,
            "s" if ib != 1 else ""
        )
    ]))


none: 19 on-target samples in 2 iteration blocks
environment: 106 on-target samples in 4 iteration blocks
body: 101 on-target samples in 2 iteration blocks
cup: 33 on-target samples in 3 iteration blocks
food: 35 on-target samples in 4 iteration blocks
nails: 69 on-target samples in 3 iteration blocks
smoke: 92 on-target samples in 3 iteration blocks
eyebrow: 235 on-target samples in 5 iteration blocks
nose: 249 on-target samples in 4 iteration blocks
above-ear: 403 on-target samples in 5 iteration blocks
behind-ear: 220 on-target samples in 4 iteration blocks
opposite-cheek: 74 on-target samples in 6 iteration blocks
chin: 166 on-target samples in 1 iteration block
cheek: 686 on-target samples in 6 iteration blocks
forehead: 176 on-target samples in 1 iteration block
top-head: 160 on-target samples in 1 iteration block
back-head: 170 on-target samples in 1 iteration block
opposite-face: 381 on-target samples in 2 iteration blocks
paint-mouth: 81 on-target samples in 3 iteration blocks

In [160]:
pilot_true = []
pilot_false = []

on_target = 'above-ear'

for row in pilot_data[(pilot_data.target == on_target) & (pilot_data.ontarget)][['thermopile1','thermopile2','thermopile3','thermopile4']].values.tolist():
    pilot_true.append({'in':row, 'out':1})
for row in pilot_data[(pilot_data.target == 'behind-ear') & (pilot_data.ontarget == False)][['thermopile1', 'thermopile2', 'thermopile3', 'thermopile4']].values.tolist():
    pilot_false.append({'in':row, 'out':0})

In [161]:
on_data = []
on_targets = []

for piles in pilot_true:
    on_data.append([float(x) for x in piles['in']])
    on_targets.append(piles['out'])

off_data = []
off_targets = []

for piles in pilot_false:
    off_data.append([float(x) for x in piles['in']])
    off_targets.append(piles['out'])

In [179]:
train_size = 50

train_data = []
train_data_p = []
test_data = []
test_data_p = []
train_targets = []
test_targets = []

train_data_p.extend(on_data[:train_size])
train_data_p.extend(off_data[:train_size])

for row in train_data_p:
    row = [np.round(float(x)/98.6,3) for x in row]
    train_data.append(row)

test_data_p.extend(on_data[train_size:])
test_data_p.extend(off_data[train_size:])

for row in test_data_p:
    row = [np.round(float(x)/98.6,3) for x in row]
    test_data.append(row)

train_targets.extend(on_targets[:train_size])
train_targets.extend(off_targets[:train_size])
test_targets.extend(on_targets[train_size:])
test_targets.extend(off_targets[train_size:])

In [217]:
clf = MLPClassifier(solver='adam', alpha=.001,
                    hidden_layer_sizes=(25,15,5), random_state=1)
clf.fit(train_data, train_targets)

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(25, 15, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [238]:
num_predict = len(test_data)

predictions = clf.predict(test_data[:num_predict])
accuracy_score(test_targets[:num_predict], predictions)

true_pos = []
true_neg = []
fals_pos = []
fals_neg = []

count_neg = len([x for x in test_targets if x == 0])
count_pos = len([x for x in test_targets if x == 1])

for sample in range(len(test_data)):
    if test_targets[sample] == 0:
        if predictions[sample] == test_targets[sample]:
            true_neg.append(sample)
        else:
            fals_neg.append(sample)
    elif test_targets[sample] == 1:
        if predictions[sample] == test_targets[sample]:
            true_pos.append(sample)
        else:
            fals_pos.append(sample)
        
fals_pos = len(fals_pos)/count_pos
fals_neg = len(fals_neg)/count_neg
true_pos = len(true_pos)/count_pos
true_neg = len(true_neg)/count_neg

print('Target: ' + str())
print('True positive rate:  ' + str(np.round(true_pos,3)))
print('True negative rate:  ' + str(np.round(true_neg,3)))
print('False positive rate: ' + str(np.round(fals_pos,3)))
print('False negative rate: ' + str(np.round(fals_neg,3)))

True positive rate:  0.779
True negative rate:  0.949
False positive rate: 0.221
False negative rate: 0.051
